In [ ]:
from google.colab import files
import os
import zipfile
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
import kaggle
print("Imported kaggle API successfully !")

Saving kaggle.json to kaggle.json
Imported kaggle API successfully !


In [ ]:
print("Downloading dataset : ")
!kaggle datasets download subirbiswas19/skin-disease-dataset
print("Downloaded Successfully!")

Dataset URL: https://www.kaggle.com/datasets/subirbiswas19/skin-disease-dataset
License(s): CC0-1.0
Downloaded Successfully!


In [ ]:
zip_ref = zipfile.ZipFile('/content/skin-disease-dataset.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNetV3Small, ResNet50
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [ ]:
import os

# Directory paths
data_dir = "/content/skin-disease-datasaet/train_set/"

# Listing the directories within the main directory
main_directory_contents = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Listing the subdirectories within each directory in the main directory
subdirectory_contents = {}
for subdir in main_directory_contents:
    subdir_path = os.path.join(data_dir, subdir)
    subdirs = [d for d in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, d))]
    if subdirs:
        subdirectory_contents[subdir] = subdirs

main_directory_contents, subdirectory_contents

(['BA- cellulitis',
  'VI-chickenpox',
  'PA-cutaneous-larva-migrans',
  'VI-shingles',
  'BA-impetigo',
  'FU-ringworm',
  'FU-nail-fungus',
  'FU-athlete-foot'],
 {})

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:

# Directory paths
data_dir = "/content/skin-disease-datasaet/train_set/"

datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.25,
    horizontal_flip=True,
    fill_mode="nearest"
)

BATCH_SIZE = 64

train_generator_original = datagen.flow_from_directory(
    data_dir,
    target_size=(192, 108),
    classes=['BA- cellulitis','BA-impetigo','FU-athlete-foot','FU-nail-fungus','FU-ringworm','PA-cutaneous-larva-migrans','VI-chickenpox','VI-shingles'],
    batch_size=BATCH_SIZE,
    subset='training'
)

validation_generator_original = datagen.flow_from_directory(
    data_dir,
    target_size=(192, 108),
    classes=['BA- cellulitis','BA-impetigo','FU-athlete-foot','FU-nail-fungus','FU-ringworm','PA-cutaneous-larva-migrans','VI-chickenpox','VI-shingles'],
    batch_size=BATCH_SIZE,
    subset='validation'
)


Found 741 images belonging to 8 classes.
Found 183 images belonging to 8 classes.


In [ ]:

def custom_generator(original_generator):
    while True:
        data, labels = next(original_generator)
        yield (tf.convert_to_tensor(data), tf.convert_to_tensor(data), tf.convert_to_tensor(data)), tf.convert_to_tensor(labels)

train_generator = custom_generator(train_generator_original)
validation_generator = custom_generator(validation_generator_original)

# Load VGG16, MobileNetV3Small, and ResNet50 models
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(192, 108, 3)))
base_model_mobilenet = MobileNetV3Small(weights='imagenet', include_top=False, input_tensor=Input(shape=(192, 108, 3)))
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(192, 108, 3)))

# Freeze the layers of all models
for model in [base_model_vgg, base_model_mobilenet, base_model_resnet]:
    for layer in model.layers[:-4]:  # Unfreeze the last 4 layers for fine-tuning
        layer.trainable = False

# Extract features using all models
features_vgg = base_model_vgg.output
features_mobilenet = base_model_mobilenet.output
features_resnet = base_model_resnet.output

# Flatten and concatenate the features
features_vgg = Flatten()(features_vgg)
features_mobilenet = Flatten()(features_mobilenet)
features_resnet = Flatten()(features_resnet)
concatenated_features = Concatenate()([features_vgg, features_mobilenet, features_resnet])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:

# Add dense layers followed by dropout with L2 regularization
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(concatenated_features)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(8, activation='softmax')(x)

# Define the hybrid model
model = Model(inputs=[base_model_vgg.input, base_model_mobilenet.input, base_model_resnet.input], outputs=predictions)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9)


In [ ]:
# Compile the model
from tensorflow.keras.metrics import Precision, Recall

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)



checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save('skincancer.h5')
model.save('skincancer.keras')


In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=120,
    steps_per_epoch=train_generator_original.samples // BATCH_SIZE, # changed steps_per_epoch
    validation_steps=validation_generator_original.samples // BATCH_SIZE, # changed validation_steps
    callbacks=[checkpoint, early_stopping]
)
# Evaluate the model on the validation set
val_loss, val_accuracy, val_precision, val_recall = model.evaluate(validation_generator,
                                                                   steps=validation_generator_original.samples // BATCH_SIZE)

# Calculate F1-score
val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-7)  # Adding a small value to avoid division by zero

# Print results
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1-score: {val_f1:.4f}")


Epoch 1/120
11/11 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.9160 - loss: 0.5404 - precision: 0.9738 - recall: 0.8648 - val_accuracy: 0.8992 - val_loss: 0.6864 - val_precision: 0.9375 - val_recall: 0.8824
Epoch 2/120
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9324 - loss: 0.5058 - precision: 0.9659 - recall: 0.8891 - val_accuracy: 0.8908 - val_loss: 0.6949 - val_precision: 0.9052 - val_recall: 0.8824
Epoch 3/120
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9189 - loss: 0.5007 - precision: 0.9525 - recall: 0.8970 - val_accuracy: 0.8992 - val_loss: 0.6747 - val_precision: 0.9352 - val_recall: 0.8487
Epoch 4/120
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 364ms/step - accuracy: 0.9274 - loss: 0.5103 - precision: 0.9510 - recall: 0.9017 - val_accuracy: 0.8739 - val_loss: 0.7818 - val_precision: 0.8938 - val_recall: 0.8487
Epoch 5/120
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 562ms/step - accuracy: 0.8986 - loss: 0.5882 - precision: 0.9371 - recall: 0.8702 - val_accuracy: 0.8487 - val_loss: 0.6